In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.utils
import torch
from torch import optim
import torch.nn.functional as F
from PIL import Image

from torch.autograd import Variable
from tqdm import tqdm 
import torch.nn as nn

In [20]:
# import imgshow and plot show
#impot class SiameseNetworkDataset 
from utils.plot_helpers import imshow, show_plot
import os

In [21]:
from torchvision.models import resnet50, ResNet50_Weights

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Preparing the data

In [23]:
class COVIDDataset(Dataset):
    """
    This class is used to create a dataset for the Siamese Network.

    Args:
        Dataset (torch.utils.data.Dataset): Pytorch Dataset class
    Returns:
        image0, image1, label    
    
    """
    def __init__(self,root_path,transform=None):
        """
        Initialize the dataset
        Args:
            imageFolderDataset (torchvision.datasets.ImageFolder): Pytorch ImageFolder class
            transform (torchvision.transforms): Pytorch transforms
        """
        self.root_path = root_path    
        self.transform = transform
        self.classes = os.listdir(root_path)
        self.classes_to_idx = {class_name:i for i, class_name in enumerate(self.classes)} 
        self.img_paths, self.labels = self._make_samples()

    def _make_samples(self):
        samples_images = []
        samples_classes = []
        for cls_name in self.classes:
            class_dir = os.path.join(self.root_path, cls_name)
            for file_name in os.listdir(class_dir):
                sample_path = os.path.join(class_dir, file_name)
                samples_images.append(sample_path)
                samples_classes.append(self.classes_to_idx[cls_name])
        return samples_images, samples_classes 

    def __getitem__(self,idx):
        path_img0 = self.img_paths[idx] 
        label0 = self.labels[idx]

        img0 = Image.open(path_img0)

        if self.transform is not None:
            img0 = self.transform(img0)
        
        return img0, torch.tensor(label0)
    
    def __len__(self):
        return len(self.img_paths)

In [24]:
# Load the training dataset

folder_datatrain = COVIDDataset(root_path=os.path.join('dataset_jpg1','train'), 
                                         transform=preprocess)

train_size = int(0.8 * len(folder_datatrain))
val_size = len(folder_datatrain) - train_size

siamese_datatrain, siamese_dataval = torch.utils.data.random_split(folder_datatrain, [train_size, val_size])


# Create a simple dataloader just for simple visualization-train
train_loader = DataLoader(siamese_datatrain,
                        shuffle=True,
                        num_workers=0,
                        batch_size=8)

# Create a simple dataloader just for simple visualization-val
val_loader = DataLoader(siamese_dataval,
                        shuffle=True,
                        num_workers=0,
                        batch_size=8)

In [38]:
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        # Initialize metrics
        train_loss = 0.0
        train_acc = 0.0
        val_loss = 0.0
        val_acc = 0.0
        num_train_batches = len(train_loader)
        num_val_batches = len(val_loader)
        min_val = 10000

        # Training loop
        model.train()
        for i, (inputs, labels) in tqdm(enumerate(train_loader), total=num_train_batches, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
            inputs = Variable(inputs).to(device)
            labels = Variable(labels).to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Update metrics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_acc += (predicted == labels).sum().item()

        # Validation loop
        model.eval()
        with torch.no_grad():
            for i, (inputs, labels) in tqdm(enumerate(val_loader), total=num_val_batches, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
                inputs = Variable(inputs).to(device)
                labels = Variable(labels).to(device)
                
                outputs = model(inputs)
            
                loss = criterion(outputs, labels)

                # Update metrics
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_acc += (predicted == labels).sum().item()

        if val_loss < min_val:
            torch.save(model.state_dict(), os.path.join('ckpt','best_model.pth'))

        # Calculate metrics
        train_loss /= num_train_batches
        train_acc /= len(train_loader.dataset)
        val_loss /= num_val_batches
        val_acc /= len(val_loader.dataset)

        # Print metrics and progress bar
        tqdm.write(f'Epoch {epoch + 1}/{num_epochs} - Training accuracy: {train_acc:.4f} - Training loss: {train_loss:.4f} - Validation accuracy: {val_acc:.4f} - Validation loss: {val_loss:.4f}')

In [39]:
lr = 0.0001
epochs = 50

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().to(device)


In [40]:
model.fc = nn.Linear(in_features = model.fc.in_features, out_features=4)

In [41]:
model.to(device);

In [42]:
train(model, train_loader, val_loader, optimizer, criterion, epochs)

Epoch 1/50 - Validation: 100%|██████████| 37/37 [00:01<00:00, 31.74it/s]


Epoch 1/50 - Training accuracy: 0.8481 - Training loss: 0.5972 - Validation accuracy: 0.9693 - Validation loss: 0.1202


Epoch 2/50 - Validation: 100%|██████████| 37/37 [00:01<00:00, 31.85it/s]


Epoch 2/50 - Training accuracy: 0.9838 - Training loss: 0.0953 - Validation accuracy: 0.9795 - Validation loss: 0.0784


Epoch 3/50 - Validation: 100%|██████████| 37/37 [00:01<00:00, 31.15it/s]


...

Epoch 48/50 - Training accuracy: 0.9974 - Training loss: 0.0052 - Validation accuracy: 0.8191 - Validation loss: 3.8430


Epoch 49/50 - Validation: 100%|██████████| 37/37 [00:01<00:00, 30.21it/s]


Epoch 49/50 - Training accuracy: 0.9974 - Training loss: 0.0037 - Validation accuracy: 0.8294 - Validation loss: 2.0143


Epoch 50/50 - Validation: 100%|██████████| 37/37 [00:01<00:00, 30.27it/s]


Epoch 50/50 - Training accuracy: 0.9983 - Training loss: 0.0033 - Validation accuracy: 0.9761 - Validation loss: 0.1684


In [43]:
# Load the training dataset

folder_datatest = COVIDDataset(root_path=os.path.join('dataset_jpg1','test'), 
                                         transform=preprocess)

# Create a simple dataloader just for simple visualization-train
test_loader = DataLoader(folder_datatest,
                        shuffle=True,
                        num_workers=0,
                        batch_size=8)

In [45]:
def test(model, loader):
    # Initialize metrics
    val_acc = 0.0
    
    model.eval()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(loader):
            inputs = Variable(inputs).to(device)
            labels = Variable(labels).to(device)
            
            outputs = model(inputs)
        
            _, predicted = torch.max(outputs.data, 1)
            val_acc += (predicted == labels).sum().item()

    val_acc /= len(val_loader.dataset)
    return val_acc


In [51]:
model.load_state_dict(torch.load(os.path.join('ckpt','best_model.pth')));

In [50]:
test_accuracy = test(model, test_loader)

print(f'Test accuracy = {round(test_accuracy,4)}')

Test accuracy = 0.5904


### Data Visualization